# Runtime -> Change Runtime -> GPU

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.3 MB/s eta 0:14:42tcmalloc: large alloc 1147494400 bytes == 0x3a288000 @  0x7fbe3b94b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:11:15tcmalloc: large alloc 1434370048 bytes == 0x7e8de000 @  0x7fbe3b94b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████▋          | 1336.2 MB 1.3 MB/s eta 0:08:19tcmalloc: large alloc 1792966656 bytes =

In [3]:
!pip install --quiet "torchvision" "torchmetrics>=0.6" "pytorch-lightning>=1.4" "ipython[notebook]" "torch>=1.6, <1.9"

     |████████████████████████████████| 408 kB 33.2 MB/s 
     |████████████████████████████████| 582 kB 72.2 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
     |████████████████████████████████| 136 kB 75.7 MB/s 
     |████████████████████████████████| 1.1 MB 45.7 MB/s 
     |████████████████████████████████| 144 kB 70.5 MB/s 
     |████████████████████████████████| 271 kB 55.1 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 


In [4]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 608 kB/s 
     |████████████████████████████████| 448 kB 71.2 MB/s 


In [5]:
import torch

torch.__version__


'1.8.1+cu111'

In [6]:
# check for the GPU provided in the runtime
!nvidia-smi
# using quiet method for controlling the log
# for suppressing the colored errors and warning in the terminal
!pip install --quiet transformers==4.1.1
# pytorch lightning for smoother model training and data loading
#!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --quiet /content/pytorch-lightning-1.6.1.tar.gz
# using HuggingFace tokenizers
!pip install --quiet tokenizers==0.9.4
# Google's sentencepiece
!pip install --quiet sentencepiece==0.1.94
# mostly pl is used while doing complex model training
import pytorch_lightning as pl
print(pl.__version__)
# argparse makes it easier to write user friendly command line interfaces
import argparse
# package for faster file name matching
import glob
# makiing directories for data 
import os
# reading json files as the data is present in json files
import json
# time module for calculating the model runtime
import time
# Allows writing status messages to a file
import logging
# generate random float numbers uniformly
import random
# regex module for text 
import re
# module provides various functions which work on 
# iterators too produce complex iterators
from itertools import chain
from string import punctuation
# pandas for data manipulation
import pandas as pd
# numpy for array operations
import numpy as np
# PyTorch
import torch
# provides various classes representing file system paths
# with appropriate semantics
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
# splitting the data 
from sklearn.model_selection import train_test_split
# ANSII color formatting for ouput in terminal
from termcolor import colored
# wrapping paragraphs into string
import textwrap
# model checkpoints in pretrained model
from pytorch_lightning.callbacks import ModelCheckpoint
'''
optimizer - AdamW
T5 Conditional Generator in which we'll give conditions
T5 tokenizer because it is fast
training the model without a learning rate
'''
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
# Seeds all the processes including numpy torch and other imported modules.
pl.seed_everything(0)
# check the version provided by Lightning
import pytorch_lightning as pl
print(pl.__version__)

Sat Apr 30 00:26:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Global seed set to 0


1.6.2
1.6.2


# Downloading the Dataset

In [7]:
#데이터 파일 로드
dat = pd.read_csv("/content/data_add_title.csv")
dat.head()

,question,context,answer_text,answer_start,answer_end,title
0,드라마 예고범의 감독은 누구일까?,예고범 일본어 予告犯 은 츠츠이 테츠야의 만화 작품이다 점프 카이 슈에이샤 에서 2...,나카무라 요시히로 히라바야시 카츠토시 사와다 메구미,6302,6332,예고범
1,심규언은 17대 지방 선거에서 몇 표를 득표하였는가?,심규언 沈圭彦 1955년 10월 1일 은 대한민국의 제17·18대 강원도 동해시장이...,20890 표,3873,3881,심규언
2,장기이식으로 인해 얻을 수 있는 효과는?,장기 이식 臟器移植 이란 어떤 조직 또는 장기의 파손된 기능을 대체할 목적으로 원래...,자유 시장이 장기 이식에서의 희소성 문제를 해결할 수 있다고 말했다,6469,6507,장기_이식
3,'망언' 논란을 야기한 천안함 사태에 대한 이윤성의 발언은 무엇이며 어떤 비판을 받...,이윤성 李允盛 1944년 10월 2일 은 前 kbs 앵커이자 前 무소속 국회의원이다...,전국 동시 지방 선거를 앞두고 있던 2010년 5월 31일 이윤성 의원은 여의도 당...,5818,6699,이윤성_(1944년)
4,2005년 행정부의 6급이하 공무원은 전체에 몇퍼정도를 차지하나요?,이윤성 李允盛 1944년 10월 2일 은 前 kbs 앵커이자 前 무소속 국회의원이다...,전체의 95정도,7770,7779,이윤성_(1944년)


In [8]:
df = dat

In [9]:
from konlpy.tag import Okt
okt = Okt()

In [10]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [11]:
MODEL_NAME ='KETI-AIR/ke-t5-base' 
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

# Creating labels for the answers
In the following cells, We have to create necessary labels for the answers. This is required so that we can extract answers accordingly to the questions.

In [12]:
class BioQADataset(Dataset):
  def __init__(
      self,
      data:pd.DataFrame,
      tokenizer:T5Tokenizer,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,
      ):
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len
  def __len__(self):
    return len(self.data)
  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]
    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_len,
      padding='max_length',
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length=self.target_max_token_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    labels = target_encoding['input_ids']
    labels[labels==0] = -100
    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )

In [13]:
train_df, val_df = train_test_split(df, test_size=0.05)

In [14]:
 class BioDataModule(pl.LightningDataModule):
   def __init__(
       self,
       train_df: pd.DataFrame,
       test_df: pd.DataFrame,
       tokenizer:T5Tokenizer,
       batch_size: int = 8,
       source_max_token_len: int = 396,
       target_max_token_len: int = 32,
       ):
     super().__init__()
     self.train_df = train_df
     self.test_df = test_df
     self.tokenizer = tokenizer
     self.batch_size = batch_size
     self.source_max_token_len = source_max_token_len
     self.target_max_token_len = target_max_token_len
   def setup(self, stage = None):     ## stage = None 추가 
     self.train_dataset = BioQADataset(
         self.train_df,
         self.tokenizer,
         self.source_max_token_len,
         self.target_max_token_len
         )
     self.test_dataset = BioQADataset(
     self.test_df,
     self.tokenizer,
     self.source_max_token_len,
     self.target_max_token_len
     )
   def train_dataloader(self):
     return DataLoader(
         self.train_dataset,
         batch_size=self.batch_size,
         shuffle=True,
         num_workers=4
         )
   def val_dataloader(self):
     return DataLoader(
         self.test_dataset,
         batch_size=self.batch_size,
         num_workers=4
         )
   def test_dataloader(self):
     return DataLoader(
         self.test_dataset,
         batch_size=1,
         num_workers=4
         )
 BATCH_SIZE = 4
 N_EPOCHS = 6
 data_module = BioDataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
 data_module.setup() 

# Building the PyTorch lightning module using T5ForConditionalGeneration model
In the following cells, we have leveraged a T5 Conditional Generator which will produce text based on some condition (this is going to be our application task – QA).

In [15]:
class BioQAModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)
    return output.loss, output.logits
  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions":outputs, "labels": labels}
  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=0.0001)
    return optimizer
model = BioQAModel()   

Downloading:   0%|          | 0.00/599 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

# Using Trainer from PyTorch-Lightning to Finetune Model Using our Dataset
We have used a trainer module to test the model and fine-tune it in the following cells. This is helpful when the model is huge, and you want to get a warmup of the model without changing the learning rate. 

In [16]:
model = BioQAModel()
# To record the best performing model using checkpoint
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)
#logger = TensorBoardLogger("training-logs", name="bio-qa")
#logger = TensorBoardLogger("training-logs", name="bio-qa")
trainer = pl.Trainer(
    #logger = logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate = 30
) 

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f0b5002d1d0>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f0b5002d1d0>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Tr

# Predictions

In [17]:
#체크포인트 파일 로드
trained_model = BioQAModel.load_from_checkpoint("/content/best-checkpoint-large-v1.ckpt")
trained_model.freeze() # 

# Generate Answers for the Questions in the Validation Set
In the following cells, we have tried and tested some questions whose answers were saved with us in the valid dataset against the predicted values given by the model. As you’ll notice the T5 model is so powerful that it will predict exact values.

In [18]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
def generate_answer(question, context):
  source_encoding=tokenizer(
      question,
      context,
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )
  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]
  return "".join(preds)

In [20]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [21]:
def isin_title(keyword):
  mask = dat[dat['title'].str.contains(keyword)]
  # if len(mask) > 0:
  #   q_def = mask
  #   return q_def
  #   # , print('자세하게 질문을 입력해주세요.')
  # else:
  key = dat[dat['context'].str.contains(keyword)]
    # use= key.drop_duplicates(['context'], ignore_index = True)
  use = pd.concat([mask, key]).drop_duplicates(['context'], ignore_index = True)
  if len(use) > 0:
    q_def = use
    return q_def
    # , print('자세하게 질문을 입력해주세요.')
  else:
    return(print('토픽에 대한 내용을 찾지 못했습니다.'))

In [22]:
def cont(t_list, question):      ### 인풋질문과 컨텍스트 코사인유사도 분석  제일 높은것 뽑아줌  
  t_list['score']=0
  t_list['score'] = t_list.apply(lambda x: cos_sim(tokenizer_t(x['context'],max_length = 6000, padding="max_length")['input_ids'],
                                      tokenizer_t(question ,max_length = 6000, padding="max_length")['input_ids']), axis=1)
  return t_list.loc[t_list['score'].idxmax()]['context']

In [23]:
def anw_test(context, question):

  source_encoding=tokenizer(
    question,
    context,
    max_length = 396,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
  generated_ids = trained_model.model.generate(
    input_ids=source_encoding["input_ids"],
    attention_mask=source_encoding["attention_mask"],
    num_beams=1,  # greedy search
    max_length=80,
    repetition_penalty=2.5,
    early_stopping=True,
    use_cache=True)
  preds = [
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids]
  return "".join(preds), context 

In [24]:
def last(question, dat):
  dat = dat.drop_duplicates(['context'], ignore_index = True)
  a = in_title(question,dat)   ## 타이틀, 컨텍스트 
  b= title_re(a, question)    ## 질문,타이틀 유사도 
  context =cosin_sim(b, question)    
  answer = anw_test(context, question)   ## 정답 예측, 컨텍스트 
  return answer

# 토크나이저 시도

In [25]:
 !git clone https://github.com/monologg/KoBERT-Transformers.git

Cloning into 'KoBERT-Transformers'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 75 (delta 27), reused 58 (delta 17), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [26]:
import sys
sys.path.append('/content/')

In [27]:
# coding=utf-8
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team and Jangwon Park
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Tokenization classes for KoBERT model """


import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {
    "vocab_file": "tokenizer_78b3253a26.model",
    "vocab_txt": "vocab.txt",
}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model",
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt",
    },
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512,
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False},
}

SPIECE_UNDERLINE = "▁"


class KoBertTokenizer(PreTrainedTokenizer):
    """
    SentencePiece based tokenizer. Peculiarities:
        - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """

    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self,
        vocab_file,
        vocab_txt,
        do_lower_case=False,
        remove_space=True,
        keep_accents=False,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        **kwargs,
    ):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, "r", encoding="utf-8") as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning(
                "You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                "pip install sentencepiece"
            )
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize("NFKD", outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text):
        """Tokenize a string."""
        text = self.preprocess_text(text)
        pieces = self.sp_model.encode(text, out_type=str)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(
                map(
                    lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0,
                    token_ids_0,
                )
            )

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """Save the sentencepiece vocabulary (copy original file) and special tokens file
        to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt



In [28]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [29]:
!pip3 install kobert-transformers

In [30]:
import kobert_transformers

In [31]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [32]:
tokenizer_t = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [33]:
!pip install tensorflow #텐서플로우 설치
!pip install sklearn #사이킷런 설치

     |████████████████████████████████| 462 kB 28.6 MB/s 


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
sentences = ("이 요리 의 레시피 를 알려줘.",
        "이 요리 어떻게 만드는 지 알려줘.")
tfidf_vectorizer = TfidfVectorizer()

# 문장 벡터화 하기(사전 만들기)
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

### 코사인 유사도 ###
from sklearn.metrics.pairwise import cosine_similarity
# 첫 번째와 두 번째 문장 비교
cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
print("코사인 유사도 측정")
print(cos_similar)

코사인 유사도 측정
[[0.41120706]]


In [35]:
def cosin_sim(t_list, question):
  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['context'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
      t_list['score'][i] = cos_similar[0][0]
  # return cos_similar[0][0], type(cos_similar[0][0])
  return t_list.loc[t_list['score'].idxmax()]['context']

In [36]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

## 정규화 ##
tfidf_normalized = tfidf_matrix/np.sum(tfidf_matrix)

##유클리디언 유사도##
euc_d_norm = euclidean_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])

In [37]:
def eu_sim(t_list, question):
  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['context'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      tfidf_normalized = tfidf_matrix/np.sum(tfidf_matrix)
      euc_d_norm = euclidean_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])
      t_list['score'][i] = cos_similar[0][0]
  # return euc_d_norm[0][0], type(euc_d_norm[0][0])
  return t_list.loc[t_list['score'].idxmin()]['context']

In [38]:
from sklearn.metrics.pairwise import manhattan_distances
manhattan_d = manhattan_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])

In [39]:
def man_sim(t_list, question):
  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['context'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      tfidf_normalized = tfidf_matrix/np.sum(tfidf_matrix)
      manhattan_d = manhattan_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])
      t_list['score'][i] = manhattan_d[0][0]
  # return euc_d_norm[0][0], type(euc_d_norm[0][0])
  return t_list.loc[t_list['score'].idxmin()]['context']

# 새 코드 

In [40]:
def in_title(question, dat):
  dat = dat.drop_duplicates(['context'], ignore_index = True)
  t_list = []
  con_list = []
  for i in range(len(dat)):
    if dat['title'][i].split('(')[0] in question.replace(' ', ''):
      contet = dat.loc[i,:]
      t_list.append(contet)

    else:
      pass
  t_list = pd.DataFrame(t_list, columns = dat.columns)
  con_list = dat[dat['context'].str.contains(okt.nouns(question)[0])]
  use = pd.concat([t_list, con_list], ignore_index=True)
  if len(use) >0:
    return use
  else :
    return print('데이터 없음')


In [41]:
def title_re(t_list, question):     

  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['question'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      tfidf_normalized = tfidf_matrix/np.sum(tfidf_matrix)
      manhattan_d = manhattan_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])
      t_list['score'][i] = manhattan_d[0][0]
  t_list = t_list[t_list['score'] == min(t_list['score'])]
  return t_list


In [42]:
def t5answer(question):
  ans = last(question, dat)
  return ans

In [43]:
# t5answer("농심을 대표하는 라면의 이름은?")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # 

('치킨라면',
 '라면은 기름에 미리 튀긴 국수를 스프와 함께 끓는 물에 넣어서 요리하는 국수 식품이다 근래에는 기름에 튀기지 않은 라면이 나왔다 보통 라면은 면과 분말스프 건더기스프가 함께 포장되어 끓는 물에 34분 정도 스프와 함께 끓이면 조리된다 라면은 밀가루와 달걀로 굵은 면을 뽑고 한 번 삶아 튀겨서 전분이 알파화한 면이 이후 뜨거운 물에 다시 먹을 수 있는 상태로 만들어지는 형태이다 면과 함께 분말스프와 건더기스프로 구성된 제품이 일반적이다 라면은 중국에서 전시 비상 식량으로 쓴 것으로 유래된다 당시 일본이 중일전쟁을 하면서 배워 왔다는 설이 있다 현재의 라면은 제2차 세계 대전이 끝난 뒤 일본에서 대만계 일본인인 안도 모모후쿠가 발명하였다고 한다 당시 미군 구호품으로 밀가루가 많이 있었기 때문에 이를 이용한 새로운 식품을 고안하게 되었다 밀가루로 국수를 만들어 기름에 튀겨 국수 안의 수분이 증발되고 이후 뜨거운 물에 들어가게 되면 본래의 상태로 풀어져 먹기 좋은 상태가 되는 것이다 최초의 즉석 라면은 1958년 8월 25일에 현재 닛신식품의 전신인 산시쇼쿠산에서 생산한 치킨라면이었다 초기 라면은 양념이 면에 더해진 형태였으나 이후 1962년에 스프를 분말로 만들고 따로 첨부한 형태의 봉지면이 인기를 끌게 되었다 1963년 9월 15일에 삼양식품 창립자 전중윤 회장은 일본의 묘조 식품으로부터 무상 기술원조를 받아 처음으로 라면을 만들어 팔기 시작했다 당시 가격은 10원이었다 처음에는 생소하여 판매가 부진했지만 무료시식회에서 그 맛을 인정받으며 곧 서민들의 음식으로 각광 받게 되었다 당시 정부의 혼분식 장려정책도 라면이 널리 보급되도록 했다 1960년대 정부는 쌀부족문제 해결을 위해 혼식 잡곡밥 과 분식 밀가루 음식 을 강제하는 정책을 실시했는데 덕분에 라면은 많이 팔릴 수밖에 없었다 그 실례로 1969년에 한 해 동안 천 오백만 봉지가 팔렸다 1970년에 들어서는 즉석 짜장면 칼국수 냉면 등의 다양한 제품이 나왔다 또 한식에 맞고 대한민국의 문화에

In [44]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 745 kB 58.1 MB/s 
     |████████████████████████████████| 53 kB 2.7 MB/s 
     |████████████████████████████████| 10.9 MB 59.6 MB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
     |████████████████████████████████| 79 kB 10.9 MB/s 
     |████████████████████████████████| 58 kB 7.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=95fe010b248b020b6869c8ab51fa5931b53b9ee4baa0726557b1cc4672402f05
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=0da53a9d82afef51b3ae0c12a213a730de3e8edf1b78e3cdcd7be1c2e6711e51
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built pyngrok python-multipart


In [45]:
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.responses import PlainTextResponse
from fastapi.responses import JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn

In [46]:
from starlette.responses import Response
# FastAPI
from fastapi.staticfiles import StaticFiles

app=FastAPI()
templates = Jinja2Templates(directory="templates")



@app.get("/")
async def root():
    return {"message": "Hello World"}

@app.post('/T5api', response_class=PlainTextResponse)
async def get_reply(request: Request, message2: str = Form(...)):
  reply = last(message2, dat)[0]
  return reply

In [47]:
#ngrok 토큰
!ngrok authtoken ngrontoken

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://6a62-35-247-145-51.ngrok.io


INFO:     Started server process [71]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying

INFO:     34.82.240.142:0 - "POST /T5api HTTP/1.1" 200 OK


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # 

INFO:     34.82.240.142:0 - "POST /T5api HTTP/1.1" 200 OK


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # 

INFO:     34.82.240.142:0 - "POST /T5api HTTP/1.1" 200 OK


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # 

INFO:     34.82.240.142:0 - "POST /T5api HTTP/1.1" 200 OK
